# Teil 5 Demo 2: Aufwertung von Daten mit Bezugsdaten

## Bibliotheken & Konfiguration

In [ ]:
packages <- c("readr", "dplyr", "tidyr", "lubridate", "ggplot2", "zoo", "forcats", "grid", "gtable")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

#install.packages("zoo")
#library(readr)
#library(dplyr)
#library(tidyr)
#library(lubridate)
#library(ggplot2)
#library(zoo)
#library(forcats)
#library(grid)
#library(gtable)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-2021-Block-7/master/Rohdaten/covid/"

## Covid19-Inzidenzdaten Mannheim laden

### Quelle

COVID-19 Datenhub des RKI beim Bundesamt für Kartographie und Geodäsie
URL: https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0
Die Daten sind die „Fallzahlen in Deutschland“ des Robert Koch-Institut (RKI) und stehen unter der Open Data Datenlizenz Deutschland – Namensnennung – Version 2.0 zur Verfügung. (https://www.govdata.de/dl-de/by-2-0)
Quellenvermerk: Robert Koch-Institut (RKI), dl-de/by-2-0

Aufbereitung für den Kurs: Filterung auf Mannheim, Auswahl relevanter Spalten, Aggregation von Teilmeldungen pro Tag

In [ ]:
mannheim.incidence.long <- read_csv2(paste(base_url, "covid_mannheim_incidence.csv", sep=""),
                                     col_types = cols(date = col_date(format = ""), category = col_character(), variable = col_character(), value = col_double()))
head(mannheim.incidence.long)

## Mannheimer Covid19-Inzidenz plotten

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
ggplot(data = mannheim.incidence.long, aes(x=date, y=value)) +
  geom_bar(stat="identity")

### Darstellung mit gleitendem 7-Tage-Durchschnitt

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
ggplot(data = mannheim.incidence.long, aes(x=date, y=rollmean(value, 7, na.pad=TRUE))) +
  geom_bar(stat="identity")

## Daten zu Maßnahmen gegen Covid19-Pandemie ergänzen

### Quelle: Covid19 Government Response Tracker der Oxford University

* Homepage: https://www.bsg.ox.ac.uk/research/research-projects/covid-19-government-response-tracker
* Github Repo: https://github.com/OxCGRT/covid-policy-tracker
    * Rohdatensatz (CSV): https://github.com/OxCGRT/covid-policy-tracker/blob/master/data/OxCGRT_latest.csv
    * Codebook: https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/codebook.md

Quellenangabe:
Oxford COVID-19 Government Response Tracker, Blavatnik School of Government, University of Oxford.

Thomas Hale , Noam Angrist , Rafael Goldszmidt, Beatriz Kira , Anna Petherick , Toby Phillips, Samuel Webster, Emily Cameron-Blake , Laura Hallas, Saptarshi Majumdar, and Helen Tatlow. (2021). “A global panel database of pandemic policies (Oxford COVID-19 Government Response Tracker).” Nature Human Behaviour.


In [ ]:
germany.response.raw <- read_csv("https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv",
                                 col_types = cols(CountryName = col_character(),CountryCode = col_character(),RegionName = col_character(),RegionCode = col_character(),Jurisdiction = col_character(),Date = col_double(),`C1_School closing` = col_double(),C1_Flag = col_double(),`C2_Workplace closing` = col_double(),C2_Flag = col_double(),`C3_Cancel public events` = col_double(),C3_Flag = col_double(),`C4_Restrictions on gatherings` = col_double(),C4_Flag = col_double(),`C5_Close public transport` = col_double(),C5_Flag = col_double(),`C6_Stay at home requirements` = col_double(),C6_Flag = col_double(),`C7_Restrictions on internal movement` = col_double(),C7_Flag = col_double(),`C8_International travel controls` = col_double(),`E1_Income support` = col_double(),E1_Flag = col_double(),`E2_Debt/contract relief` = col_double(),`E3_Fiscal measures` = col_double(),`E4_International support` = col_double(),`H1_Public information campaigns` = col_double(),H1_Flag = col_double(),`H2_Testing policy` = col_double(),`H3_Contact tracing` = col_double(),`H4_Emergency investment in healthcare` = col_double(),`H5_Investment in vaccines` = col_double(),`H6_Facial Coverings` = col_double(),H6_Flag = col_double(),`H7_Vaccination policy` = col_double(),H7_Flag = col_double(),`H8_Protection of elderly people` = col_double(),H8_Flag = col_double(),M1_Wildcard = col_character(),ConfirmedCases = col_double(),ConfirmedDeaths = col_double(),StringencyIndex = col_double(),StringencyIndexForDisplay = col_double(),StringencyLegacyIndex = col_double(),StringencyLegacyIndexForDisplay = col_double(),GovernmentResponseIndex = col_double(),GovernmentResponseIndexForDisplay = col_double(),ContainmentHealthIndex = col_double(),ContainmentHealthIndexForDisplay = col_double(),EconomicSupportIndex = col_double(),EconomicSupportIndexForDisplay = col_double()))
head(germany.response.raw %>% filter(CountryName == "Germany", grepl("202101", Date)))

## Beispielhafte Konvertierung von 3 Attributen in ein Long-Format

In [ ]:
germany.response.long <- pivot_longer(germany.response.raw %>%
             filter(CountryName == "Germany") %>% # Filterung auf Deutschland
             select(Date, `C1_School closing`, `C4_Restrictions on gatherings`, `C6_Stay at home requirements`), # benötigte Spalten selektieren
             !Date, # Spalte(n), die "im rechten Teil" pivotiert werden sollen. 
                    # Vorsicht Trick: hier wird mit dem Ausrufezeichen "negiert", d.h. wir geben die (eine) Spalte an, 
                    # die "im linken Teil" stehenbleiben soll
             names_to = "variable", # Spalte, in die die Namen der pivotierten Spalten überführt werden sollen
             values_to = "value"    # Spalte, in die die Inhalte der pivotierten Spalten überführt werden sollen
)
germany.response.long$Date <- ymd(germany.response.long$Date) # Text in Date-Spalte in echtes Datum konvertieren
germany.response.long$category <- "response" # Spalte mit Datenkategorie ergänzen
germany.response.long <- germany.response.long %>% rename(date = Date) # Date-Spalte umbenennen, um sie mit den Inzidenzdaten zu vereinheitlichen

head(germany.response.long %>% filter(grepl("2021-01", date)), 10)

## Maßnahmen plotten

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
ggplot(data = germany.response.long, aes(x=date, y=value, fill=variable)) +
  geom_bar(stat="identity") +
  facet_wrap(~variable, ncol=1)

## Inzidenz & Maßnahmen zusammenführen & gemeinsam plotten

In [ ]:
# Teildatensätze "hintereinander" zusammenführen (vgl. SQL UNION Statement)
mannheim.all.long <- bind_rows(mannheim.incidence.long, germany.response.long)

# Reihenfolge der Plots definieren
mannheim.all.long$variable <- factor(mannheim.all.long$variable, levels = c("incidence", "C1_School closing", "C4_Restrictions on gatherings", "C6_Stay at home requirements"))

# Daten in gemeinsamem Plot darstellen
ggplot(data = mannheim.all.long) +
  # Inzidenz-Plot (Darstellung mit gleitendendem Mittelwert)
  geom_bar(data = mannheim.all.long %>% filter(category %in% c("incidence")), aes(x=date, y=rollmean(value, 7, na.pad=TRUE), fill=variable), stat="identity") +
  # Maßnahmen-Plots (direkte Darstellung der Werte ohne gleitenden Mittelwert)
  geom_bar(data = mannheim.all.long %>% filter(category %in% c("response")), aes(x=date, y=value, fill=variable), stat="identity") +
  # Verteilung der Kurven n auf getrennte Plots
  facet_grid(fct_rev(variable) ~ ., scales="free_y") +
  # Darstellung der Plot-Titel rechts und nicht hochkant
  theme(strip.text.y.right = element_text(angle = 0)) +
  # Ausblenden der Legende
  guides(fill=FALSE)

## Mobilitätsdaten hinzufügen

### Quelle: Apple Covid19 Mobilitätstrends

* Homepage: https://covid19.apple.com/mobility
    * siehe dort für Nutzungsbedingungen (kein direkter Link verfügbar), Download der Rohdaten
    
Daten liegen in "wide"-Format mit einer Zeile pro Verkehrsmittel (Laufen, ÖPNV, Auto) und je einer Spalte pro Tag vor, aufgelöst für Deutschland bis auf Kreise (also 3 Zeilen von Kreis)

Aufbereitung für den Kurs: Filterung auf Mannheim, Selektion relevanter Spalten, Verschiebung der Skale von 100 +/- Änderung auf 0 +/- Änderung, Pivotierung in "long"-Format

In [ ]:
mannheim.mobility.long <- read_csv2(paste(base_url, "covid_mannheim_mobility.csv", sep=""),
                                     col_types = cols(date = col_date(format = ""), category = col_character(), variable = col_character(), value = col_double()))
head(mannheim.mobility.long)

## Inzidenz-, Maßnahmen & Mobilitätsdaten gemeinsam plotten

In [ ]:
# Teildatensätze "hintereinander" zusammenführen (vgl. SQL UNION Statement)
mannheim.all.long <- bind_rows(mannheim.incidence.long, germany.response.long, mannheim.mobility.long)

# Reihenfolge der Plots definieren
mannheim.all.long$variable <- factor(mannheim.all.long$variable, levels = c("incidence", "walking", "transit", "driving", "C1_School closing", "C4_Restrictions on gatherings", "C6_Stay at home requirements"))

# Daten in gemeinsamem Plot darstellen
options(repr.plot.width = 7, repr.plot.height = 5)
plot <- ggplot(data = mannheim.all.long) +
  # Inzidenz- & Mobility-Plots (Darstellung mit gleitendendem Mittelwert)
  geom_bar(data = mannheim.all.long %>% filter(category %in% c("incidence", "mobility")), aes(x=date, y=rollmean(value, 7, na.pad=TRUE), fill=variable), stat="identity") +
  # Maßnahmen-Plots (direkte Darstellung der Werte ohne gleitenden Mittelwert)
  geom_bar(data = mannheim.all.long %>% filter(category %in% c("response")), aes(x=date, y=value, fill=variable), stat="identity") +
  # Verteilung der Kurven auf getrennte Plots
  facet_grid(fct_rev(variable) ~ ., scales="free_y") +
  # Darstellung der Plot-Titel rechts und nicht hochkant, kleine Achsenbeschriftung
  theme(strip.text.y.right = element_text(angle = 0), axis.text.y = element_text(size=4)) +
  # Ausblenden der Legende
  guides(fill=FALSE)

# Größen der Plots anpassen
gt = ggplot_gtable(ggplot_build(plot))
gt$heights[7]  = 0.5*gt$heights[7]
gt$heights[9]  = 0.5*gt$heights[9]
gt$heights[11] = 0.5*gt$heights[11]
gt$heights[19] = 5  *gt$heights[19]
grid.draw(gt)